# Final act scraper

## 1. Library and data import

In [11]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import ast
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import time

In [12]:
df = pd.read_csv('final_sample_cod_TA_2025.csv')

In [13]:
df

,Unnamed: 0,url,reference,title,subjects,key_players,key_events,documentation_gateway,transparency,final_act,...,Stage reached in procedure,Committee dossier,Mandatory consultation of other institutions,Other legal basis,legislative_proposal_url,legislative_proposal_text,decisions_url_list,resolution_text_1,position_text_1,text_adopted_others_1
0,0,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0255(COD),Justice programme 2028-2034,7.40.02 Judicial cooperation in civil and comm...,{'European Parliament': [{'Joint committee res...,{'Legislative proposal published': {'Date': '0...,"[{'Institution': 'European Commission', 'Docum...",NaN,NaN,...,Awaiting committee decision,CJ03/10/03804,NaN,NaN,https://eur-lex.europa.eu/legal-content/EN/TXT...,"EUROPEAN COMMISSION Brussels, 3.9.2025 COM(2...",NaN,NaN,NaN,NaN
1,1,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0405(COD),Placing on the market of genetically modified ...,"3.10.09.06 Agro-genetics, GMOs, 4.60.02 Consum...",{'European Parliament': [{'Joint committee res...,{'Legislative proposal published': {'Date': '1...,"[{'Institution': 'European Commission', 'Docum...",NaN,NaN,...,Preparatory phase in Parliament,NaN,[{'text': 'European Economic and Social Commit...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0540(COD),"Union support for asylum, migration and integr...",7.10 Free movement and integration of third-co...,{'European Parliament': [{'Committee responsib...,{'Legislative proposal published': {'Date': '1...,"[{'Institution': 'European Commission', 'Docum...",{'Other Members': [{'Name': [{'text': 'ASENS L...,NaN,...,Awaiting committee decision,LIBE/10/03652,NaN,NaN,https://eur-lex.europa.eu/legal-content/EN/TXT...,"EUROPEAN COMMISSION Brussels, 16.7.2025 COM(...",NaN,NaN,NaN,NaN
3,3,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0550(COD),'AgoraEU' programme 2028–2034,"1.20 Citizen's rights, 4.10.03 Child protectio...",{'European Parliament': [{'Joint committee res...,{'Legislative proposal published': {'Date': '1...,"[{'Institution': 'European Commission', 'Docum...","{'Rapporteurs, Shadow Rapporteurs and Committe...",NaN,...,Awaiting committee decision,CJ65/10/03702,[{'text': 'European Economic and Social Commit...,NaN,https://eur-lex.europa.eu/legal-content/EN/TXT...,"EUROPEAN COMMISSION Brussels, 16.7.2025 COM(...",NaN,NaN,NaN,NaN
4,4,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0227(COD),Global Europe,"6.20 Common commercial policy in general, 8.70...",{'European Parliament': [{'Joint committee res...,{'Legislative proposal published': {'Date': '1...,"[{'Institution': 'European Commission', 'Docum...",NaN,NaN,...,Awaiting committee decision,CJ19/10/03697,"[{'text': 'European Court of Auditors', 'href'...",NaN,https://eur-lex.europa.eu/legal-content/EN/TXT...,"EUROPEAN COMMISSION Brussels, 16.7.2025 COM(...",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,114,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0232(COD),Protection of workers from the risks related t...,"4.15.15 Health and safety at work, occupationa...",{'European Parliament': [{'Committee responsib...,{'Legislative proposal published': {'Date': '1...,"[{'Institution': 'European Commission', 'Docum...","{'Rapporteurs, Shadow Rapporteurs and Committe...",NaN,...,Awaiting committee decision,EMPL/10/03700,[{'text': 'European Economic and Social Commit...,NaN,https://eur-lex.europa.eu/legal-content/EN/TXT...,"EUROPEAN COMMISSION Brussels, 18.7.2025 COM(...",NaN,NaN,NaN,NaN
115,115,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0074(COD),Extension of the timeframe for the establishme...,"7.30.30 Action to combat crime, 7.40.04 Judici...",{'European Parliament': [{'Committee responsib...,{'Legislative proposal published': {'Date': '0...,"[{'Institution': 'European Commission', 'Docum...",NaN,{'pdfFinal act': '/oeil/en/procedure-file/pdf?...,...,Procedure completed,

## 2. Scraping the final acts

In [ ]:
def construire_lien_html(row_str):
    # Convert string to dict
    try:
        d = ast.literal_eval(row_str)
    except Exception:
        return None
    
    # Get the 2nd value
    valeurs = list(d.values())
    if len(valeurs) < 2:
        return None
    url = valeurs[1]

    # Extract the block of type 2025R8528
    m = re.search(r"(\d{4}[A-Za-z]\d{4})", url)
    if not m:
        return None
    
    bloc = m.group(1)          # e.g.: "2025R8528"
    
    # Replace the letter with 0
    bloc_modifie = bloc[:4] + "0" + bloc[5:]   # e.g.: "202508528"

    # Build the final EUR-Lex link
    final_url = f"https://eur-lex.europa.eu/legal-content/EN/TXT/HTML/?uri=OJ:L_{bloc_modifie}"
    
    return final_url

In [ ]:
def get_eurlex_text_from_html(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    eli_container = soup.find('div', class_='eli-container')
    
    if not eli_container:
        return "[Error: Main document container not found.]"

    full_text_lines = []

    # 1. Title
    title_div = eli_container.find('div', id='tit_1')
    if title_div:
        for p in title_div.find_all('p', class_='oj-doc-ti'):
            full_text_lines.append(p.get_text(strip=True))

    # 2. Text body: Iterating over key structures
    # Instead of a global find_all, we look for table rows (recitals) 
    # and isolated paragraphs (articles/titles)
    
    # Targeting elements that contain useful text
    elements = eli_container.find_all(['tr', 'p'], recursive=True)

    for element in elements:
        # Skip if it is a note
        if any(cls in element.get('class', []) for cls in ['oj-note', 'footnote']):
            continue

        # CASE A: It is a table row (often a recital)
        if element.name == 'tr':
            cells = element.find_all('td')
            if len(cells) >= 2:
                # cell[0] contains (1), cell[1] contains the text
                num = cells[0].get_text(strip=True)
                val = cells[1].get_text(strip=True)
                if num and val:
                    full_text_lines.append(f"{num} {val}")
                    continue # Skip to next element to avoid duplicates

        # CASE B: It is an article title
        if 'oj-ti-art' in element.get('class', []):
            full_text_lines.append(f"\n{element.get_text(strip=True)}\n")
        
        # CASE C: Normal paragraph but not already inside a table
        # (To avoid capturing recital text a second time)
        elif 'oj-normal' in element.get('class', []) and not element.find_parent('td'):
            text = element.get_text(strip=True)
            if text:
                full_text_lines.append(text)

    return '\n'.join(dict.fromkeys(full_text_lines)) # dict.fromkeys removes duplicates while preserving order

In [16]:
# Initialize your columns
df['final_act_text'] = None
df['final_act_url'] = None

# Initialize the Selenium driver once
driver = webdriver.Chrome()

# Loop with progress bar
for idx, row in df.iterrows() :

    print(f'Processing row {idx} out of {len(df)}')

    if pd.notna(row['final_act']):
        # Construct URL
        df.at[idx, 'final_act_url'] = construire_lien_html(row['final_act'])
        url = df.at[idx, 'final_act_url']

        try:
            driver.get(url)

            # Wait for the page body to load (replace with a more specific element if needed)
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.TAG_NAME, "body"))
            )

            time.sleep(0.5)

            # Get page source and scrape
            html_content = driver.page_source
            scraped_text = get_eurlex_text_from_html(html_content)

            # Save scraped text
            df.at[idx, 'final_act_text'] = scraped_text

        except Exception as e:
            print(f"Error processing row {idx}: {e}")
            df.at[idx, 'final_act_text'] = None

# Quit the driver after finishing all rows
driver.quit()

Processing row 0 out of 119
Processing row 1 out of 119
Processing row 2 out of 119
Processing row 3 out of 119
Processing row 4 out of 119
Processing row 5 out of 119
Processing row 6 out of 119
Processing row 7 out of 119
Processing row 8 out of 119
Processing row 9 out of 119
Processing row 10 out of 119
Processing row 11 out of 119
Processing row 12 out of 119
Processing row 13 out of 119
Error processing row 13: Message: invalid argument: 'url' must be a string
  (Session info: chrome=143.0.7499.170)
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x7ff7c8b788e5
	0x7ff7c8b78940
	0x7ff7c895165d
	0x7ff7c89fbb9a
	0x7ff7c89d1fda
	0x7ff7c89fac97
	0x7ff7c899ac29
	0x7ff7c899ba93
	0x7ff7c8e90640
	0x7ff7c8e8af80
	0x7ff7c8ea96e6
	0x7ff7c8b95de4
	0x7ff7c8b9ed8c
	0x7ff7c8b82004
	0x7ff7c8b821b5
	0x7ff7c8b67ee2
	0x7ffe5d97e8d7
	0x7ffe5ee4c53c

Processing row 14 out of 119
Processing row 15 out of 119
Processing row 16 out of 119
Processing row 17 out of 119
Processing row 18 o

Here, it looks like there is an error but it is not the case : it means the Parliament has adopted a final act, but they haven't linked the document to their page, making it not reachable

In [17]:
df

,Unnamed: 0,url,reference,title,subjects,key_players,key_events,documentation_gateway,transparency,final_act,...,Mandatory consultation of other institutions,Other legal basis,legislative_proposal_url,legislative_proposal_text,decisions_url_list,resolution_text_1,position_text_1,text_adopted_others_1,final_act_text,final_act_url
0,0,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0255(COD),Justice programme 2028-2034,7.40.02 Judicial cooperation in civil and comm...,{'European Parliament': [{'Joint committee res...,{'Legislative proposal published': {'Date': '0...,"[{'Institution': 'European Commission', 'Docum...",NaN,NaN,...,NaN,NaN,https://eur-lex.europa.eu/legal-content/EN/TXT...,"EUROPEAN COMMISSION Brussels, 3.9.2025 COM(2...",NaN,NaN,NaN,NaN,None,None
1,1,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0405(COD),Placing on the market of genetically modified ...,"3.10.09.06 Agro-genetics, GMOs, 4.60.02 Consum...",{'European Parliament': [{'Joint committee res...,{'Legislative proposal published': {'Date': '1...,"[{'Institution': 'European Commission', 'Docum...",NaN,NaN,...,[{'text': 'European Economic and Social Commit...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
2,2,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0540(COD),"Union support for asylum, migration and integr...",7.10 Free movement and integration of third-co...,{'European Parliament': [{'Committee responsib...,{'Legislative proposal published': {'Date': '1...,"[{'Institution': 'European Commission', 'Docum...",{'Other Members': [{'Name': [{'text': 'ASENS L...,NaN,...,NaN,NaN,https://eur-lex.europa.eu/legal-content/EN/TXT...,"EUROPEAN COMMISSION Brussels, 16.7.2025 COM(...",NaN,NaN,NaN,NaN,None,None
3,3,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0550(COD),'AgoraEU' programme 2028–2034,"1.20 Citizen's rights, 4.10.03 Child protectio...",{'European Parliament': [{'Joint committee res...,{'Legislative proposal published': {'Date': '1...,"[{'Institution': 'European Commission', 'Docum...","{'Rapporteurs, Shadow Rapporteurs and Committe...",NaN,...,[{'text': 'European Economic and Social Commit...,NaN,https://eur-lex.europa.eu/legal-content/EN/TXT...,"EUROPEAN COMMISSION Brussels, 16.7.2025 COM(...",NaN,NaN,NaN,NaN,None,None
4,4,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0227(COD),Global Europe,"6.20 Common commercial policy in general, 8.70...",{'European Parliament': [{'Joint committee res...,{'Legislative proposal published': {'Date': '1...,"[{'Institution': 'European Commission', 'Docum...",NaN,NaN,...,"[{'text': 'European Court of Auditors', 'href'...",NaN,https://eur-lex.europa.eu/legal-content/EN/TXT...,"EUROPEAN COMMISSION Brussels, 16.7.2025 COM(...",NaN,NaN,NaN,NaN,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,114,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0232(COD),Protection of workers from the risks related t...,"4.15.15 Health and safety at work, occupationa...",{'European Parliament': [{'Committee responsib...,{'Legislative proposal published': {'Date': '1...,"[{'Institution': 'European Commission', 'Docum...","{'Rapporteurs, Shadow Rapporteurs and Committe...",NaN,...,[{'text': 'European Economic and Social Commit...,NaN,https://eur-lex.europa.eu/legal-content/EN/TXT...,"EUROPEAN COMMISSION Brussels, 18.7.2025 COM(...",NaN,NaN,NaN,NaN,None,None
115,115,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0074(COD),Extension of the timeframe for the establishme...,"7.30.30 Action to combat crime, 7.40.04 Judici...",{'European Parliament': [{'Committee responsib...,{'Legislative proposal published': {'Date': '0...,"[{'Institution': 'European Commission', 'Docum...",NaN,{'pdfFinal act': '/oeil/en/procedure-file/pdf?...,...,NaN,Rules of Procedure EP 165,https://eur-lex.europa.eu/legal-content/EN/TXT...,"EUROPEAN COMMISSION Brussels, 2.4.2025 COM(2...",['https://www.europarl.europa.eu/doceo/documen...,European Parliament legislative resolution

In [18]:
df['final_act'][118]

nan

In [19]:
df = df.drop(columns=['Unnamed: 0'])

In [20]:
# Save the data
df.to_csv('final_sample_cod_TA_final_act_2025.csv')